<a href="https://colab.research.google.com/github/JackPerry0/DeterminantCalculator/blob/main/slamAlg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from XRPLib.defaults import *
import time
import math
import random


# available variables from defaults: left_motor, right_motor, drivetrain,
#      imu, rangefinder, reflectance, servo_one, board, webserver
# Write your code Here


#establish the stepping size.
#In other words what distance does each
#Position in an array represent.
"""
start = time.time()
while True:
    print(rangefinder.distance())
    time.sleep(1)
    if time.time() - start > 5:
        break

"""

#Using the method above I conclude that the ultrasonic sensor measures in cm
#the length of the robot is roughly 23cm. Therefore each block/element in the
#array should be maybe 8cm, or rougly one third of the robot.

#Want to utilize this method to scan the surroundings however we may want to slow down this function
#drivetrain.turn(360)

#Here is the function


#Our map is going to have three values -->
# 0 = drivable terrain
# -1 = unknown/when we find an wall but we dont know its depth so we put -1 behind what its blocking us from viewing.
# 10 = where our bot is located.

#We can establish a max boundary just by doing a 360 and finding the longest range this is certainly not optimal,
#we should be able to adjust/add to the array by moving

#Lets start by establishing a base array.
#just do multiple small turns and measure


def slopeFromAngle(degree):

    rad = math.radians(degree)
    slope = math.tan(rad)

    return slope

#print(establishMap())
#np.zeros((max/3, max/3), dtype=int)

#test the array
#print(establishMap)

#What if we over shot and did like every 11 degrees and did that a bunch of times at different lenghts.
#our bot struggles to do micro movements like 1 degree turns

#or we can try to use time

def DistanceForDegree():

    head = imu.get_heading()
    head = int(head)
    default = head


    start = time.time()
    #print("default", default)
    #print("head", head)
    drivetrain.set_effort(-.25, .25)
    map = [0 for i in range(361)]
    while default != (head) or (time.time() - start) < 1:
        default = imu.get_heading()
        default_rounded = round(default)
        dist = int(rangefinder.distance())
        if map[default_rounded] == 0:
            if dist == 65535:   #this means our sensor could not reach this distance in other words to far for our sensor. so we will replace these with the max length.
                map[default_rounded] = -1
            else:
                map[default_rounded] = dist
        default = int(default)
    drivetrain.stop()



    return map

#DistanceForDegree()

def place_robot(map_grid, robot_size, map_size):
    #print("map_size", map_size)

    width, height = map_size
    center_x, center_y = width // 2, height // 2  # Find map center

    half_size = robot_size // 2  # Half the size of the robot

    # Loop over the 7x7 grid area
    for dx in range(-half_size, half_size + 1):
        for dy in range(-half_size, half_size + 1):
            x = center_x + dx
            y = center_y + dy
            #print(x)
            #print(y)
            map_grid[y][x] = 2  # Mark robot area

    return map_grid

#squishes the array by a factor x on the width and height. There is a priority to values when there are conflicting values between the x*x block
# Priority: 2,5,0,-1. So if there is one 2 in the block then in the squashed matrix the value will be 2 regardless of whats in the other boxes.
def squashMap(original_map, squash_factor=3):


    original_height = len(original_map)
    original_width = len(original_map[0])

    new_height = original_height // squash_factor
    new_width = original_width // squash_factor

    squashed_map = [[-1 for _ in range(new_width)] for _ in range(new_height)]

    for new_y in range(new_height):
        for new_x in range(new_width):

            block_values = []
            for dy in range(squash_factor):
                for dx in range(squash_factor):
                    orig_y = new_y * squash_factor + dy
                    orig_x = new_x * squash_factor + dx
                    block_values.append(original_map[orig_y][orig_x])

            if 2 in block_values:
                squashed_map[new_y][new_x] = 2
            elif 5 in block_values:
                squashed_map[new_y][new_x] = 5
            elif 0 in block_values:
                squashed_map[new_y][new_x] = 0
            else:
                squashed_map[new_y][new_x] = -1

    return squashed_map

def find_optimal_scaling_factor(max_distance, map_size, target_robot_size):
    # Initial guess for scaling_factor
    scaling_factor = 1.0
    tolerance = 1e-6  # Small threshold for convergence
    max_iterations = 1000

    for _ in range(max_iterations):
        # Calculate robot size based on current scaling_factor
        robot_size = target_robot_size / scaling_factor

        # Calculate new scaling_factor based on robot_size
        new_scaling_factor = max_distance / (map_size // 2 - robot_size)

        # Check if the difference is within tolerance
        if abs(new_scaling_factor - scaling_factor) < tolerance:
            return new_scaling_factor

        # Update scaling_factor for the next iteration
        scaling_factor = new_scaling_factor

    raise RuntimeError("Failed to converge to a solution")

def print_map_distances(map_distances, map_size):
    red = "\033[91m"
    RESET = "\033[0m"
    for angle, distance in enumerate(map_distances):
        if distance > map_size:
            print(f"{red}angle {angle} distance {distance}{RESET}")
        else:
            print(f"angle {angle}  distance {distance}")

def createMap(map_size=100, _Scale_Factor= -1):
    time.sleep(3)

    #DEGREES
    map_distances = DistanceForDegree()
    max_distance = max(map_distances)

    if _Scale_Factor == -1:
        #scaling_factor = max_distance / (map_size//2 - robot_size) # but robot_size = 21/scaling_factor --> this cannot be solved algebraically.
        scaling_factor = find_optimal_scaling_factor(max_distance, map_size, 21)
        #max_distance must squeeze into that dimension
    else:
        scaling_factor = _Scale_Factor

    #standardizing the map distances to our map size.
    map_distances = [distance / scaling_factor for distance in map_distances]

    #map_size = max(map_distances) * 2 + 21 #to account for robot size

    #for testing
    #map_of_cords = [[0,0] for _ in range(len(map_distances))]
    #map_of_free = [(0,0)]

    #print("before map_distances", map_distances)

    for angle in range(len(map_distances)):

        if map_distances[angle] == -1:   #THIS NEEDS TO BE CHANGED THIS SHOULD JUST RESULT IN ALL ZEROS AND NO 5 INDICATOR THEREFORE WE SHOULD FIX TEH LOGIC IN BUILDING THE MAP SUCH THAT ITS NOT ALWAYS REQUIRED TO HAVE A 5 IN ORDER TO ENCODE THE MAP WITH SOMETHING OTHER THAN -1.
            map_distances[angle] = map_size + 1 #result in the obstacle being outside the map and therefore not place.

    #print_map_distances(map_distances, map_size)
    #print("map_distances", map_distances)

    #creating the map. Each element represents 1 cm x 1cm.
    map = [[-1 for _ in range(map_size)] for _ in range(map_size)]

    #establishing the robots position in the center.
    # 2 denotes our robot. the robot is 21cm x 21 cm. Sow we need to create a 7by7 surrounding the center.

    map = place_robot(map, (int(21/scaling_factor)), (map_size, map_size))  # NEED TO FIND ROBOTS SIZE RELATIVE TO GRID
    robot_x = map_size //2
    robot_y = map_size //2

    for angle in range(360):
        distance = map_distances[angle]

        if distance == 0:
            continue



        angle_rad = math.radians(angle)
        obstacle_x = int(robot_x + distance* math.cos(angle_rad))
        obstacle_y = int(robot_y + distance * math.sin(angle_rad))

        #for testing
        #map_of_cords[angle][0] = obstacle_x
        #map_of_cords[angle][1] = obstacle_y

        if 0 <= obstacle_x < map_size and 0 <= obstacle_y < map_size:
            map[obstacle_y][obstacle_x] = 5  # Mark obstacle as '5'
        #else: #verification that the map is correct.
        #   print(f"angle {angle}  distance {distance}")
        for step in range(1, distance):
            free_x = int(robot_x + step * math.cos(angle_rad))
            free_y = int(robot_y + step * math.sin(angle_rad))

            if 0 <= free_x < map_size and 0 <= free_y < map_size:
                if map[free_y][free_x] == -1:  # Update only unknown cells
                    map[free_y][free_x] = 0
                    #for testing
                    #map_of_free = [(free_x, free_y)] + map_of_free

    #print("map_size", map_size)
    #print("dimensions y", len(map))
    #print("dimensions x", len(map[0]))

    view_color_map(map) #to view the map before squashing.
    print("squashed map:")
    map = squashMap(map)

    #for testing
    #print("map_of_cords", map_of_cords)
    #print("map_of_free", map_of_free)

    return map

def view_color_map(map):
    # ANSI escape codes for colors
    COLORS = {
        -1: "\033[37m",  # White for unknown regions
        0: "\033[34m",   # Blue for navigable areas
        2: "\033[32m",   # Green for the robot's position
        5: "\033[31m",   # Red for obstacles
        "RESET": "\033[0m"  # Reset color
    }

    for row in map:
        # Format each row with color based on value
        formatted_row = " ".join(
            f"{COLORS.get(num, COLORS['RESET'])}{num:>3}{COLORS['RESET']}"
            for num in row
        )
        print(formatted_row)

def view_black_white(map):
    cell_width = 4  # Fixed width for each cell, including padding

    # Create a uniform grid
    for row in map:
        formatted_row = "".join(f"{num:>{cell_width}}" for num in row)
        print(formatted_row)


def find_perimeter_zeros(grid):

    if not grid or not grid[0]:
        return []  # Return empty if the grid is empty

    rows = len(grid)
    cols = len(grid[0])
    perimeter_coords = []

    # Check the first and last rows
    for col in range(cols):
        if grid[0][col] == 0:  # Top row
            perimeter_coords.append((0, col))
        if grid[rows - 1][col] == 0:  # Bottom row
            perimeter_coords.append((rows - 1, col))

    # Check the first and last columns (excluding corners already added)
    for row in range(1, rows - 1):
        if grid[row][0] == 0:  # Left column
            perimeter_coords.append((row, 0))
        if grid[row][cols - 1] == 0:  # Right column
            perimeter_coords.append((row, cols - 1))

    return perimeter_coords


def q_learning(grid, objectives=[(0,0)], risk_factor=1, episodes=5000, alpha=0.1, gamma=0.9, epsilon=0.1):

    rows, cols = len(grid), len(grid[0])
    actions = ["UP", "DOWN", "LEFT", "RIGHT"]  # Action space
    q_table = {}  # Q-values for each state-action pair

    # Initialize the robot size
    robot_size = 1.0  # Assume it starts with a unit size

    # Helper to determine next state and reward
    def next_state_reward(x, y, action):
        dx, dy = 0, 0
        if action == "UP":
            dx, dy = -1, 0
        elif action == "DOWN":
            dx, dy = 1, 0
        elif action == "LEFT":
            dx, dy = 0, -1
        elif action == "RIGHT":
            dx, dy = 0, 1

        # New position
        new_x, new_y = x + dx, y + dy
        if not (0 <= new_x < rows and 0 <= new_y < cols):  # Out of bounds
            return (x, y), -1

        # Rewards based on grid content
        cell_value = grid[new_x][new_y]
        reward = -1  # Default reward for moving
        if cell_value == 5:
            reward = -1000
        elif cell_value == 2:
            reward = -risk_factor
        elif (new_x, new_y) in objectives and grid[new_x][new_y] != 5:
            reward = 1000

        return (new_x, new_y), reward

    # Q-learning iterations
    for _ in range(episodes):
        # Fixed starting position (center of the grid)
        x, y = rows // 2, cols // 2

        while True:
            # Represent state as a tuple (position and size)
            state = (x, y, round(robot_size, 2))
            if state not in q_table:
                q_table[state] = {action: 0 for action in actions}  # Initialize Q-values for state

            # Choose action (epsilon-greedy)
            if random.uniform(0, 1) < epsilon:
                action = random.choice(actions)  # Random action
            else:
                action = max(q_table[state], key=q_table[state].get)  # Best action from Q-table

            # Transition to next state
            (new_x, new_y), reward = next_state_reward(x, y, action)

            # Update robot size if stepping on `2`
            if grid[new_x][new_y] == 2:
                robot_size /= risk_factor  # Reduce size

            # Represent the next state
            next_state = (new_x, new_y, round(robot_size, 2))
            if next_state not in q_table:
                q_table[next_state] = {action: 0 for action in actions}

            # Update Q-value using the Q-learning formula
            old_value = q_table[state][action]
            next_max = min(q_table[next_state].values())  # Min reward for multi-square occupation
            q_table[state][action] = old_value + alpha * (reward + gamma * next_max - old_value)

            # Break if reached an objective
            if (new_x, new_y) in objectives and grid[new_x][new_y] != 5:
                break

            # Transition to the next state
            x, y = new_x, new_y

    return q_table



#print("\033[31mThis should be red text\033[0m")
_map = createMap(_Scale_Factor=1)
view_black_white(_map)

_objectives = find_perimeter_zeros(_map)

#policy = q_learning(grid=_map, risk_factor=1,objectives=_objectives )
#print(policy)



